In [2]:
import sys
sys.path.append(r'/Users/malwersh/Desktop')
import MyFCA
import time
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

import random
import numpy as np
import time
import numpy as np
from scipy.sparse import random
from scipy.sparse import coo_matrix
from scipy.sparse import coo_matrix, csr_matrix, csc_matrix
from sklearn.cluster import KMeans
from sklearn.metrics import davies_bouldin_score, silhouette_score, silhouette_samples
import matplotlib.pyplot as plt
from scipy.sparse import csr_matrix
from scipy.sparse import random, csr_matrix
import numpy as np
from scipy.sparse import random
import heapq
# Load the categorical attribute data into a pandas DataFrame
#data = pd.read_csv(r'/Users/malwersh/Desktop/bunchdatsets/balance-scale/balance-scalecopy.csv')
data = pd.read_csv(r'/Users/malwersh/Desktop/bunchdatsets/tae/taescopy.csv')
#data = pd.read_csv(r'/Users/malwersh/Desktop/bunchdatsets/breast-cancer1/breastCancerss.csv')
#data = pd.read_csv(r'/Users/malwersh/Desktop/bunchdatsets/CarEvaluation/carcopy.csv')
#data = pd.read_csv(r'/Users/malwersh/Downloads/lenses/lenses1.csv')


In [3]:
# converting the dataset to formal context

import pandas as pd

# Create a list of unique values for each column
unique_values = {}
for column in data.columns:
    unique_values[column] = data[column].unique()

# Create an empty binary matrix with the appropriate number of rows and columns
num_instances = len(data)
num_columns = sum([len(values) for values in unique_values.values()])
binary_matrix = pd.DataFrame(0, index=range(num_instances), columns=range(num_columns))

# Rename the columns of the binary matrix based on the unique values
column_index = 0
column_mapping = {}
attributes_names=[]
for column, values in unique_values.items():
    for value in values:
        binary_matrix.loc[:, column_index] = (data[column] == value).astype(int)
        column_mapping[column_index] = f"{column}_{str(value)}"
        attributes_names.append(column_mapping[column_index])
        column_index += 1

binary_matrix = binary_matrix.rename(columns=column_mapping)

# Print the binary matrix
print(binary_matrix)


     Eng_Nat_spk_1  Eng_Nat_spk_2  Course_Instructor_23  Course_Instructor_15  \
0                1              0                     1                     0   
1                0              1                     0                     1   
2                1              0                     1                     0   
3                1              0                     0                     0   
4                0              1                     0                     0   
..             ...            ...                   ...                   ...   
146              0              1                     0                     0   
147              0              1                     0                     0   
148              1              0                     0                     0   
149              0              1                     0                     0   
150              0              1                     0                     0   

     Course_Instructor_5  C

In [4]:
density = binary_matrix.mean().mean()
print('Density:', density)

Density: 0.04950495049504949


In [5]:
data=binary_matrix.values


In [9]:
# sometime I use reduce_density to reduce the generated formal concepts
def reduce_density(data, reduction_factor=0.30):
    flat_data = data.flatten()
    non_zero_indices = np.nonzero(flat_data)[0]
    num_to_zero = int(len(non_zero_indices) * reduction_factor)
    zero_indices = np.random.choice(non_zero_indices, num_to_zero, replace=False)
    flat_data[zero_indices] = 0
    return flat_data.reshape(data.shape)

reduced_density_data = reduce_density(data)

non_zero_count = np.count_nonzero(reduced_density_data)
total_elements = reduced_density_data.size
# Calculating the density
density = non_zero_count / total_elements
density

0.03468625008196184

In [10]:
data=reduced_density_data

In [6]:


class RandomContext:
    def __init__(self, num_objects, num_attributes, dens):
        self.num_objects = num_objects
        self.num_attributes = num_attributes 
        self.dens = dens

    def _generate_set(self, num, prefix):
        return [f"{prefix}{i + 1}" for i in range(num)]

    def get_objects(self):
        return self._generate_set(self.num_objects, 'o')
    
    def get_attributes(self):
        return self._generate_set(self.num_attributes, 'a')
    
    def get_context(self):
        return random(self.num_objects, self.num_attributes, density=self.dens, format='csr', data_rvs=np.ones)

class FormalConcepts:
    def __init__(self, objects, attributes, context):
        self.objects = objects
        self.attributes = attributes
        self.context = context
        self.concepts = self._get_formal_concepts()

    def _extract_indices(self, elements, reference_list):
        return [reference_list.index(e) for e in elements]

    def _set_with_mask(self, mask, reference_list):
        return {reference_list[i] for i, m in enumerate(mask) if m}

    def objects_with_attributes(self, subset_Y):
        subset_indices = self._extract_indices(subset_Y, self.attributes)
        mask = np.all(self.context[:, subset_indices].toarray(), axis=1)
        return self._set_with_mask(mask, self.objects)

    def attributes_with_objects(self, subset_O):
        subset_indices = self._extract_indices(subset_O, self.objects)
        mask = np.all(self.context[subset_indices, :].toarray(), axis=0)
        return self._set_with_mask(mask, self.attributes)

    def _get_formal_concepts(self):
        concepts = []
        B = set(self.attributes_with_objects(self.objects_with_attributes(())))
        A = set(self.objects_with_attributes(set(B)))
        concepts.append((A,B))
        while B != set(self.attributes):
            for index, i in reversed(list(enumerate(self.attributes))):
                if i in B:
                    B.remove(i)    
                else:
                    I_plus = self.attributes[:index]
                    inter_Un = B | set((i,)) # Use the `|` operator to compute the union
                    B_p = set(self.objects_with_attributes(inter_Un))
                    B_plus = set(self.attributes_with_objects(B_p))
                    if not set(I_plus) & (B_plus - B): # Use the `&` operator to compute the intersection
                        B = B_plus
                        D = tuple(B)
                        A = set(self.objects_with_attributes(B))
                        concepts.append((A,D))
                        break
        return  sorted(concepts, key=lambda x: len(x[0]))


    def object_concept(self):
        object_concept = {}
        for o in self.objects:
            shared_att=self.attributes_with_objects((o,))
            shared_obj=self.objects_with_attributes(shared_att)
            C_a=(set(shared_obj),set(shared_att))
            for idx, c in enumerate(self.concepts):
                if C_a[0].issubset(c[0]) and C_a[1].issubset(set(c[1])):
                    if idx in object_concept:
                        object_concept[idx] |= {o}
                    else:
                        object_concept[idx] = {o}
        return object_concept

    def attribute_concept(self):
        attribute_concept = {}
        for a in self.attributes:
            shared_obj=self.objects_with_attributes((a,))
            shared_att=self.attributes_with_objects(shared_obj)
            C_a=(set(shared_obj),set(shared_att))
            for idx, c in enumerate(self.concepts):
                if C_a[0].issubset(c[0]) and C_a[1].issubset(set(c[1])):
                    if idx in attribute_concept:
                        attribute_concept[idx] |= {a}
                    else:
                        attribute_concept[idx] = {a}
        return attribute_concept

    def _conceptualize(self, elements, func):
        element_concept = {}
        for elem in elements:
            shared_set = func((elem,))
            C_a = (shared_set, func(shared_set))
            for idx, c in enumerate(self.concepts):
                if C_a[0].issubset(c[0]) and C_a[1].issubset(c[1]):
                    element_concept.setdefault(idx, set()).add(elem)
        return element_concept

class LatticeBuilder:
    def __init__(self, concepts):
        self.concepts = concepts
    def build_lattice(self):
        accumulation = {frozenset(i[1]): set() for i in self.concepts}
        lattice_edges={frozenset(i[1]): [] for i in reversed(self.concepts)}
        border = [self.concepts[-1][-1]]
        concepts_sorted = sorted(self.concepts, key=lambda x: len(x[1]))
        lattice_idx = []
        lattice=[]
        relations=[]
        for concept in concepts_sorted[1:]:
            candidateSet = {frozenset(concept[1]) & set(B) for B in border}
            for candidate in candidateSet:
                if not accumulation[candidate] & set(concept[1]):
                    lattice_members=(set(concept[1]), set(candidate))
                    lattice.append(lattice_members)
                    lattice_edges[frozenset(concept[1])].append(frozenset(candidate))
                    lattice_edges[frozenset(candidate)].append(frozenset(concept[1]))
                    if candidate not in border:
                        border.append(candidate)
                    lattice_idx.append((border.index(candidate),concepts_sorted.index(concept)))
                    relations.append((candidate,concept[1]))
                    accumulation[candidate] |= set(concept[1]) - candidate
            border.append(frozenset(concept[1]))
        
        return lattice_idx,lattice,lattice_edges, relations
class LatticeBuilderKernel:
    def __init__(self, concepts):
        self.concepts = concepts
        print(len(self.concepts))
    def build_lattice(self):
        accumulation = {frozenset(i[1]): set() for i in self.concepts}
        lattice_edges={frozenset(i[1]): [] for i in reversed(self.concepts)}
        border = [self.concepts[-1][-1]]
        concepts_sorted = sorted(self.concepts, key=lambda x: len(x[1]))
        lattice_idx = []
        lattice=[]
        for concept in concepts_sorted[1:]:
            candidateSet = {frozenset(concept[1]) & set(B) for B in border}
            for candidate in candidateSet:
                if candidate in accumulation:
                    if not accumulation[candidate] & set(concept[1]):
                        lattice_members=(set(concept[1]), set(candidate))
                        lattice.append(lattice_members)
                        lattice_edges[frozenset(concept[1])].append(frozenset(candidate))
                        lattice_edges[frozenset(candidate)].append(frozenset(concept[1]))
                        if candidate not in border:
                            border.append(candidate)
                        lattice_idx.append((border.index(candidate),concepts_sorted.index(concept)))
                        accumulation[candidate] |= set(concept[1]) - candidate
            border.append(frozenset(concept[1]))
        
        return lattice_idx,lattice,lattice_edges, relations  


In [13]:
#generating random context
context = RandomContext(1200, 60, 0.10)
new_context=context.get_context()
Nobjects=context.get_objects()
Nattr=context.get_attributes()


In [7]:
#using the loaded data to generate the concepts
new_context=data
context=csr_matrix(new_context)
attributes=attributes_names
#number of objects
objects=[f"{i}" for i in range(151)]

fc = FormalConcepts(objects, attributes, context)
concepts=fc._get_formal_concepts()
print('Done')
Lattice_Builder=LatticeBuilder(concepts)
lattice_idx,lattice,lattice_edges,relations=Lattice_Builder.build_lattice()
print(len(lattice))
print(len(lattice_edges))

Done
619
276


In [14]:
print(len(attributes))


101


In [10]:
concepts

[(set(),
  ('Class_Size_16',
   'Class_Size_28',
   'Course_14',
   'Course_Instructor_18',
   'Course_2',
   'Course_17',
   'Class_Size_18',
   'Course_Instructor_15',
   'Class_Size_13',
   'Class_Size_26',
   'Summer_or_regular_2',
   'Course_8',
   'Course_Instructor_14',
   'Course_Instructor_13',
   'Course_15',
   'Class_Size_51',
   'Course_23',
   'Class_Size_33',
   'Class_Size_10',
   'Class_Size_17',
   'Class_Size_44',
   'Course_12',
   'Eng_Nat_spk_1',
   'Class_Size_55',
   'Class_Size_34',
   'Class_Size_66',
   'Course_Instructor_8',
   'Class_Size_22',
   'Class_Size_54',
   'Course_Instructor_9',
   'Course_25',
   'Class_Size_14',
   'Course_Instructor_19',
   'Course_Instructor_20',
   'Class_Size_46',
   'Class_Size_42',
   'Class_Size_58',
   'Class_Size_36',
   'Course_Instructor_21',
   'Course_Instructor_5',
   'Course_Instructor_4',
   'Course_Instructor_7',
   'Course_Instructor_11',
   'Class_Size_48',
   'Class_Size_7',
   'Class_Size_19',
   'Class_Size

In [157]:
# shorest path algorithm

from typing import Dict, List, Union, Tuple
import heapq

def shortest_path_costLL(start: frozenset, end: frozenset, 
                         lattice_edges: Dict[frozenset, List[frozenset]], 
                         up_cost: int, down_cost: int) -> Tuple[Union[int, None], Union[List[set], None]]:
    """
    Calculate the shortest path cost in a lattice from start to end.
    """
    # Initialize cost map and queue
    cost_map = {node: (float('inf'), None) for node in lattice_edges.keys()}
    cost_map[start] = (0, None)
    queue = [(0, start)]
    
    visited = set()
    
    while queue:
        current_cost, current_node = heapq.heappop(queue)
        
        if current_node == end:
            path = []
            while current_node is not None:
                path.append(set(current_node))
                current_node = cost_map[frozenset(current_node)][1]
            return current_cost, path
        
        visited.add(frozenset(current_node))
        
        neighbors = lattice_edges.get(frozenset(current_node), [])
        
        for neighbor in neighbors:
            if set(neighbor) not in visited:
                cost = up_cost if current_node.issuperset(neighbor) else down_cost
                total_cost = current_cost + cost
                if total_cost < cost_map[frozenset(neighbor)][0]:
                    cost_map[frozenset(neighbor)] = (total_cost, current_node)
                    heapq.heappush(queue, (total_cost, neighbor))
    
    return None, None

In [389]:
concepts[3][1]

('Class_Size_37',
 'Summer_or_regular_2',
 'Course_2',
 'Course_Instructor_3',
 'Eng_Nat_spk_2')

In [386]:
concepts[200][1]

('Course_Instructor_13', 'Course_3', 'Summer_or_regular_1')

In [660]:
# test the shortest path

# Define up_cost and down_cost as per your old code
up_cost = 2
down_cost = 1

# Define start and end nodes as per your old code
start=  frozenset({'Course_Instructor_13', 'Course_3', 'Summer_or_regular_1'}) 
end=frozenset({'Eng_Nat_spk_2', 'Course_3', 'Summer_or_regular_1'})
''
# Call the refactored function
import time

s = time.time()
MinCost, path = shortest_path_costLL(start, end,lattice_edges, up_cost, down_cost)
e = time.time()

print(e - s)
print(f'Shortest path cost from {set(start)} to {set(end)} is: {MinCost}')
print(f'Steps to reach end node: {path}')

0.013536930084228516
Shortest path cost from {'Course_Instructor_13', 'Course_3', 'Summer_or_regular_1'} to {'Eng_Nat_spk_2', 'Course_3', 'Summer_or_regular_1'} is: 3
Steps to reach end node: [{'Eng_Nat_spk_2', 'Course_3', 'Summer_or_regular_1'}, {'Course_Instructor_13', 'Eng_Nat_spk_2', 'Course_3', 'Summer_or_regular_1'}, {'Course_Instructor_13', 'Course_3', 'Summer_or_regular_1'}]


In [23]:
from typing import Set, List, Tuple, Dict, Union, Optional
def find_ancestors(concept: frozenset, concepts: List[Tuple[frozenset, frozenset]]) -> List[frozenset]:
    ancestors = []
    for _, intent in concepts:
        if intent.issubset(concept):
            ancestors.append(intent)
    return ancestors

def find_descendants(concept: frozenset, concepts: List[Tuple[frozenset, frozenset]]) -> List[frozenset]:
    descendants = []
    for _, intent in concepts:
        if concept.issubset(intent):
            descendants.append(intent)
    return descendants

# Generate the ancestor and descendant dictionaries for all concepts
def generate_concept_ancestors(concepts: List[Tuple[frozenset, frozenset]]) -> Dict[frozenset, List[frozenset]]:
    concept_ancestors = {}
    for _, intent in concepts:
        concept_ancestors[intent] = find_ancestors(intent, concepts)
    return concept_ancestors

def generate_concept_descendants(concepts: List[Tuple[frozenset, frozenset]]) -> Dict[frozenset, List[frozenset]]:
    concept_descendants = {}
    for _, intent in concepts:
        concept_descendants[intent] = find_descendants(intent, concepts)
    return concept_descendants



# Convert the concepts to frozensets for immutability and hashability
converted_concepts = [(frozenset(extent), frozenset(intent)) for extent, intent in concepts]
A=frozenset({'Course_3', 'Summer_or_regular_1', 'Course_Instructor_15', 'Class_Size_17', 'Eng_Nat_spk_2'})
B=frozenset({'Class_Size_19', 'Course_3', 'Summer_or_regular_1', 'Course_Instructor_23', 'Eng_Nat_spk_1'})
# Generate the ancestor and descendant dictionaries for all concepts
concept_ancestors = generate_concept_ancestors(converted_concepts)
concept_descendants = generate_concept_descendants(converted_concepts)
print('done')
# Run the function to find the closest common ancestor and descendant along with their respective costs
start=time.time()

end=time.time()
print(end-start)


done
2.09808349609375e-05


In [24]:
# Function to build a sub-lattice containing only relevant nodes and edges between A and B
def build_sub_lattice(A: frozenset, B: frozenset, 
                      ancestors_dict: Dict[frozenset, List[frozenset]], 
                      descendants_dict: Dict[frozenset, List[frozenset]], 
                      full_lattice_edges: Dict[frozenset, List[frozenset]]) -> Dict[frozenset, List[frozenset]]:
    
    # Initialize a set with A and B
    all_relevant_nodes = set([A, B])
    
    # Add ancestors and descendants of A and B
    all_relevant_nodes.update(ancestors_dict.get(A, []))
    all_relevant_nodes.update(descendants_dict.get(A, []))
    all_relevant_nodes.update(ancestors_dict.get(B, []))
    all_relevant_nodes.update(descendants_dict.get(B, []))
    # Create a sub-lattice containing only the edges between relevant nodes
    sub_lattice = {node: [neighbor for neighbor in full_lattice_edges.get(node, []) if neighbor in all_relevant_nodes] 
                   for node in all_relevant_nodes}

    return sub_lattice

In [882]:
import random
from collections import Counter

# Initialize a counter
counter = Counter()

# Assign random frequencies to each concept in the lattice
for concept in lattice_edges.keys():
    counter[concept] = random.randint(1, 1000)  # Assigning random frequency between 1 and 100

# Output the full counter to check the random frequencies
#print("Full Counter:")
#print(counter)

# Filter the counter based on a condition, for example, frequency > 50
threshold = 1
filtered_counter = {k: v for k, v in counter.items() if v > threshold}

# Output the filtered counter
#print("Filtered Counter:")
#print((filtered_counter))

In [883]:
len(filtered_counter)

33

In [884]:
from collections import Counter
import heapq
import logging

logging.basicConfig(level=logging.INFO)

class ConceptAnalyzer:
    def __init__(self,converted_concept,concept_ancestors,concept_descendants,concept, lattice, up_cost=2, down_cost=1):
        self.lattice = lattice
        self.converted_concepts=converted_concepts
        self.up_cost = up_cost
        self.down_cost = down_cost
        self.core_set = set()
        self.cost_memo = {}  # To memoize cost calculations
        self.concept_ancestors=concept_ancestors
        self.concept_descendants=concept_descendants
        self.concept=concept

    def calculate_aggregated_derivation_cost(self, core_set, filtered_counter):
        """Calculate the aggregated derivation cost based on the given core set."""
        total_cost = 0
        for concept, frequency in filtered_counter.items():
            min_cost = self.find_min_cost(concept, core_set)
            total_cost += frequency * min_cost
        return total_cost
    
    def find_min_cost(self, concept, core_set):
        # Check if the concept is already in the core_set
        if concept in core_set:
            return 0

        # Initialize the minimum cost to infinity
        min_cost = float('inf')

        for core_concept in core_set:
            # Use memoization to check if this cost has already been calculated
            if (concept, core_concept) in self.cost_memo:
                cost = self.cost_memo[(concept, core_concept)]
            else:
                # If not, calculate it
                sub_lattice = build_sub_lattice(
                    concept, core_concept, 
                    self.concept_ancestors, 
                    self.concept_descendants, 
                    self.lattice
                )
                #print('sub_lattice',len(sub_lattice))
                #ss=time.time()
                cost, _ = shortest_path_costLL(concept, core_concept, sub_lattice,self.up_cost,self.down_cost)
                # Store the calculated cost in the memoization dictionary
                #ee=time.time()
                #print(ee-ss)
                self.cost_memo[(concept, core_concept)] = cost

            # Update the minimum cost
            if cost < min_cost:
                min_cost = cost
        return min_cost

    
    def find_core_set(self, filtered_counter, max_core_concepts):
        """Find the core set of concepts."""
        candidates = list(self.lattice.keys())
        self.core_set.clear()
        T_concept=frozenset(self.concept[0][1])
        B_concept=frozenset(self.concept[-1][1])
        #print(T_concept,B_concept)
        
        while len(self.core_set) < max_core_concepts:

            best_cost, best_candidate = float("inf"), None
            for candidate in candidates:
                if candidate != T_concept and candidate != B_concept:
                    temp_core_set = self.core_set | {candidate}
                    current_cost = self.calculate_aggregated_derivation_cost(temp_core_set, filtered_counter)

                    if current_cost < best_cost:
                        best_cost, best_candidate = current_cost, candidate

            if best_candidate:
                self.core_set.add(best_candidate)
                candidates.remove(best_candidate)
            if len(self.core_set) >= max_core_concepts:
                break
        #print(best_cost, best_candidate)
        return list(self.core_set)


In [885]:
s = time.time()
converted_concepts = [(frozenset(extent), frozenset(intent)) for extent, intent in concepts]
concept_ancestors = generate_concept_ancestors(converted_concepts)
concept_descendants = generate_concept_descendants(converted_concepts)

# Initialize the ConceptAnalyzer class
concept_analyzer = ConceptAnalyzer(converted_concepts,concept_ancestors,concept_descendants,concepts, lattice_edges)

# Find the core set
max_core_concepts =2 #round(len(concepts)*0.20)
core_set = concept_analyzer.find_core_set(filtered_counter, max_core_concepts)

# Calculate the aggregated derivation cost based on the core set
total_cost = concept_analyzer.calculate_aggregated_derivation_cost(core_set, filtered_counter)
e = time.time()

# Output the core set and total cost
print(f"Core set: {core_set}")
print(f"Total aggregated derivation cost: {total_cost}")
print('Time:',e - s)

Core set: [frozenset({'predator', 'herbivore', 'middle', 'walk', 'wild', 'domestic', '4_legs'}), frozenset({'small', 'walk', 'predator', 'domestic', '4_legs'})]
Total aggregated derivation cost: 39686
Time: 0.07647991180419922


In [808]:
def precompute_distances(converted_concepts,concept_ancestors,concept_descendants,lattice):
    matrix = []
    i=1
    for node_a in converted_concepts:
        row = []
        for node_b in converted_concepts:
            # If not, calculate it
            sub_lattice = build_sub_lattice(
                    node_a, node_b, 
                    concept_ancestors, 
                    concept_descendants, 
                    lattice)
            # Assuming the first element of the tuple is the distance
            distance = shortest_path_costLL(node_a, node_b, sub_lattice, up_cost=2, down_cost=1)[0]
            row.append(distance)
        matrix.append(row)
        i=i+1
    return matrix

converted_concepts = [(frozenset(extent), frozenset(intent)) for extent, intent in concepts]
concept_ancestors = generate_concept_ancestors(converted_concepts)
concept_descendants = generate_concept_descendants(converted_concepts)

converted_concepts = list(lattice_edges.keys())[1:-1]
distance_matrix = precompute_distances(converted_concepts,concept_ancestors,concept_descendants,lattice_edges)

In [795]:
import pickle
import numpy as np
def import_distance_matrix(filename):
    with open(filename, 'rb') as file:
        matrix = pickle.load(file)
    return matrix

# Example usage
loaded_distance_matrix = import_distance_matrix('distance_matrix_Tae.pkl')

distance_matrix=loaded_distance_matrix
import pickle
import numpy as np

In [796]:

import pickle

def export_distance_matrix(matrix, filename):
    with open(filename, 'wb') as file:
        pickle.dump(matrix, file)

# Example usage
export_distance_matrix(distance_matrix, 'distance_matrix_cancerdataset2.pkl')


In [809]:
len(distance_matrix)

31

In [869]:
    def shortest_path_totest(start, end):
        if not lattice_edges:
            return None

        cost_map = {node: (float('inf'), None) for node in list(lattice_edges.keys())}
        cost_map[start] = (0, None)
        queue = [(0, start)]

        #if visited is None:
        visited = set()

        while queue:
            current_cost, current_node = heapq.heappop(queue)
            if current_node == end:
                path = []
                while current_node is not None:
                    path.append(set(current_node))
                    current_node = cost_map[frozenset(current_node)][1]
                return current_cost

            visited.add(frozenset(current_node))
            try:
                neighbors = lattice_edges[frozenset(current_node)]
            except KeyError:
                # Handle the error here, for example:
                print("Current node not found in lattice_edges dictionary.")
                neighbors = []
            for neighbor in neighbors:
                if set(neighbor) not in visited:
                    #cost = up_cost if (current_node, neighbor) in lattice else down_cost
                    cost= up_cost if current_node.issuperset(neighbor) else down_cost
                    total_cost = current_cost + cost
                    if total_cost < cost_map[frozenset(neighbor)][0]:
                        cost_map[frozenset(neighbor)] = (total_cost, current_node)
                        heapq.heappush(queue, (total_cost, neighbor))

        return None

In [886]:
nodes = list(lattice_edges.keys())

def assign_clusters(centroids_n):
    clusters_n = {centroid: [] for centroid in centroids_n}
    for node in nodes:
        if node==set():
            continue
        min_distance = float('inf')
        closet_centroid = None
        for centroid in centroids_n:
            dist = shortest_path_totest(node, centroid)
            if dist < min_distance:
                min_distance = dist
                closet_centroid = centroid
        clusters_n[closet_centroid].append(node)
    return clusters_n

cluster_n=assign_clusters(core_set)

In [887]:
from sklearn.metrics import silhouette_score

# Assuming `dist_matrix` is  computed distance matrix and `labels` are  cluster labels
labels = [None] * len(nodes)
cluster=0
for centroid, cluster_nodes in cluster_n.items():
    for node in cluster_nodes:
        labels[nodes.index(node)] = cluster
    cluster=cluster+1
labels = labels[1:-1]
len(labels)

31

In [888]:
object_concepts=fc.object_concept()

In [889]:
# Assuming clusters is supposed to be a list of lists, where each list represents the objects associated with a centroid.
clusters = [[],[]]
cluster_index=0

for centroid, members in cluster_n.items():
    # Ensure centroid is a valid index for clusters list
    for member in members:
        index = nodes.index(member)  # Get the index of the member in nodes list
        if index in object_concepts:  # Check if this index has a corresponding object in object_concepts
            # Append the object from object_concepts to the correct cluster
            clusters[cluster_index].append(object_concepts[index])
    cluster_index+=1

# Output the clusters list to check the contents
print(clusters)

[[{'lion'}, {'goat'}, {'cat'}, {'sparrow', 'finch'}], [{'horse'}, {'large_dog'}, {'pig'}, {'small_dog'}, {'eagle'}]]


In [890]:
object_concepts=fc.object_concept()
# Assuming clusters is supposed to be a list of lists, where each list represents the objects associated with a centroid.
clusters = [[],[]]
cluster_index=0

for centroid, members in cluster_n.items():
    # Ensure centroid is a valid index for clusters list
    for member in members:
        index = nodes.index(member)  # Get the index of the member in nodes list
        if index in object_concepts:  # Check if this index has a corresponding object in object_concepts
            # Append the object from object_concepts to the correct cluster
            clusters[cluster_index].append(index)
    cluster_index+=1

# Output the clusters list to check the contents
print(clusters)

[[2, 11, 12, 13], [1, 3, 4, 5, 6]]


In [891]:
len(clusters[0])

4

In [892]:
len(clusters[1])

5

In [893]:
d_labels = {}  # Use a dictionary to map node indices to their cluster labels 

cluster = 0
for cluster_nodes in clusters:
    print(len(cluster_nodes))
    for node in cluster_nodes:
        d_labels[node] = cluster  # Set the cluster label for each node
    cluster += 1


# Output labels list to check
print(d_labels)


4
5
{2: 0, 11: 0, 12: 0, 13: 0, 1: 1, 3: 1, 4: 1, 5: 1, 6: 1}


In [894]:
# Create an empty array to hold labels, filled with some placeholder value (e.g., -1)
# The size of the array is determined by the maximum key in your dictionary
max_index = max(d_labels.keys())
corrected_labels = np.full(max_index + 1, -1)  # Assuming indices start from 0 and are contiguous

# Now fill in the correct labels from the dictionary
for point, cluster in d_labels.items():
    corrected_labels[point] = cluster

In [895]:
# Given the corrected labels array and the original distance matrix

original_distance_matrix = np.array(distance_matrix)# (your full distance matrix)

# Exclude unassigned points
assigned_indices = np.where(corrected_labels != -1)[0]
assigned_labels = corrected_labels[assigned_indices]
assigned_distance_matrix = original_distance_matrix[assigned_indices][:, assigned_indices]
print(assigned_labels)



[1 0 1 1 1 1 0 0 0]


In [896]:
import numpy as np
from sklearn.metrics import silhouette_score

# Step 1: Calculate the center of the plot
center = np.mean(distance_matrix, axis=0)

# Step 2: Calculate the distance of each data point from the center
distances = np.linalg.norm(distance_matrix - center, axis=1)

# Step 3: Create a new axis
new_axis = np.expand_dims(distances, axis=1)
new_matrix = np.hstack((distance_matrix, new_axis))

# Step 4: Check linear separability (visualization)
import numpy as np
from sklearn.metrics import silhouette_score

# Compute silhouette score
score = silhouette_score(new_matrix, labels)
print('Silhouette Score: ', score)

Silhouette Score:  0.14267530820668914


In [897]:
from sklearn.metrics import davies_bouldin_score

dbi_score = davies_bouldin_score(distance_matrix, labels)
print('Davies-Bouldin Index:', dbi_score)

Davies-Bouldin Index: 2.0853053846094864


In [786]:
from sklearn.metrics import silhouette_samples, silhouette_score

# silhouette_samples function
silhouette_vals = silhouette_samples(distance_matrix, labels, metric='precomputed')
print("Silhouette scores from scikit-learn:", silhouette_vals)


Silhouette scores from scikit-learn: [ 0.37006173  0.285       0.30632411  0.43701149  0.3222029  -0.0562753
 -0.07730934  0.2764058  -0.00147929  0.44719626  0.40423032  0.20885284
 -0.0278481   0.14604592  0.02312139 -0.05208106  0.12118343  0.15467172
  0.11234729 -0.12389381  0.1         0.05547344  0.16813049 -0.160941
 -0.05309735  0.05113901  0.09483794  0.17779151  0.00884956  0.15868996
  0.27695876 -0.1386715   0.22157203  0.11585607  0.10451306  0.20296943
  0.08450704 -0.04440311  0.1797619   0.10618597  0.17918261  0.03534644
 -0.02923491  0.09353741 -0.16682411  0.31370558  0.17331671  0.06624826
 -0.07446809  0.17345679  0.13483731  0.1875     -0.04423268  0.1294143
  0.05493221  0.05113901  0.06071099  0.13388676  0.07229196  0.22419355
  0.05002305  0.0861678   0.18321456  0.06884562  0.06754875  0.15247785
  0.19399169  0.05509259  0.05509259  0.0560166   0.10212515  0.13900067
  0.21422087  0.25309165  0.20909492  0.18104601  0.06363636  0.06754875
  0.06500692  0.08

In [803]:
context=[[0,1,1,0,0,0,1,1,0,1,0,1],
[0,1,0,1,0,0,1,1,0,1,0,1],
[0,1,1,0,0,0,1,1,0,1,0,0],
[0,1,0,0,1,1,0,1,0,1,1,0],
[0,1,0,1,0,1,1,1,0,1,1,0],
[1,0,1,0,0,1,0,0,1,0,1,0],
[1,0,1,0,0,1,0,0,1,0,1,0],
[1,0,1,0,0,0,1,0,1,0,1,0],
[0,1,0,1,0,0,1,1,0,0,1,1],
[0,1,0,1,0,1,0,1,0,1,1,0]]
context=csr_matrix(context)
context
attributes=['2_legs', '4_legs', 'small', 'middle', 'large','herbivore', 'predator', 'walk', 'fly', 'domestic', 'wild', 'bite']
objects=['small_dog','large_dog','cat','horse','pig','finch','sparrow','eagle','lion','goat']
fc = FormalConcepts(objects, attributes, context)

fc = FormalConcepts(objects, attributes, context)
concepts=fc._get_formal_concepts()
print('Done')
Lattice_Builder=LatticeBuilder(concepts)
lattice_idx,lattice,lattice_edges,relations=Lattice_Builder.build_lattice()
print(len(lattice))
print(len(lattice_edges))


Done
67
33
